# Обучение и оптимизация модуля балансировки нагрузки врачей (Load Balancer)

**Цель:** Перераспределять слоты между врачами для оптимальной загрузки и минимизации времени ожидания.


## 1. Импорт библиотек и подключение к базе данных PostgreSQL (medical_db)

In [1]:
import os, json, warnings, logging
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')

from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError
from scipy.optimize import minimize

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
LOG = logging.getLogger('LoadBalancer')

# Настройки подключения к локальной базе medical_db
DB_CONFIG = {
    'host': 'localhost',
    'database': 'medical_db',
    'user': 'postgres',
    'password': 'postgres',
    'port': '5432'
}
engine = create_engine(f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")
print("Подключение к medical_db установлено")

Matplotlib is building the font cache; this may take a moment.


Подключение к medical_db установлено


## 2. Загрузка и подготовка данных (PLANNING, MEDECINS, MOTCONSU, DIR_ANSW)

In [2]:
# Пример запроса расписания на 14 дней вперёд
query = """
SELECT 
    p.PLANNING_ID, p.DATE_CONS, p.HEURE, p.MEDECINS_CREATOR_ID, p.PATIENTS_ID,
    p.CANCELLED, m.SPECIALISATION_ID, m.FM_DEP_ID,
    mc.VID_PRIEMA, mc.CONS_DURATION,
    da.IS_APPLIED
FROM PLANNING p
LEFT JOIN MEDECINS m ON p.MEDECINS_CREATOR_ID = m.MEDECINS_ID
LEFT JOIN MOTCONSU mc ON p.PLANNING_ID = mc.PLANNING_ID
LEFT JOIN DIR_ANSW da ON p.PLANNING_ID = da.PLANNING_ID
WHERE p.DATE_CONS BETWEEN CURRENT_DATE AND CURRENT_DATE + INTERVAL '14 days'
"""
df = pd.read_sql(query, engine)
print(f'Загружено {len(df)} записей')
df.head()

OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

## 3. Feature Engineering: загрузка, время ожидания, специализация, совместимость

In [3]:
# Пример расчёта загрузки врача
doctor_load = df.groupby('MEDECINS_CREATOR_ID').size().reset_index(name='slots_count')
doctor_load['load_percent'] = doctor_load['slots_count'] / 20  # 20 слотов в день — норма
df = df.merge(doctor_load, on='MEDECINS_CREATOR_ID', how='left')

# Время ожидания (заглушка)
df['wait_time'] = np.random.randint(5, 60, size=len(df))

# Совместимость по специализации
df['is_speciality_match'] = (df['SPECIALISATION_ID'].notna()).astype(int)

df.head()

NameError: name 'df' is not defined

## 4. Оптимизационная постановка (MIP/GA)
- min sum |ρ_d-0.8| + β*WaitTime
- Ограничения: дневная норма, специализация, окна


In [ ]:
# Пример оптимизации с помощью scipy (или ortools, pymoo для GA)
from scipy.optimize import minimize

def objective(x, loads, wait_times, alpha=0.7, beta=0.3):
    # x — распределение пациентов по врачам (0/1)
    load_term = np.sum(np.abs(loads + x - 0.8))
    wait_term = np.sum(wait_times * x)
    return alpha * load_term + beta * wait_term

# Заглушка: оптимизация для одного врача
loads = df['load_percent'].values
wait_times = df['wait_time'].values
x0 = np.zeros_like(loads)
res = minimize(objective, x0, args=(loads, wait_times), method='Powell')
print('Результат оптимизации:', res.fun)

## 5. Оценка метрик и сохранение результатов

In [ ]:
# Оценка средней загрузки и времени ожидания
mean_load = df['load_percent'].mean()
mean_wait = df['wait_time'].mean()
print(f'Средняя загрузка: {mean_load:.2f}, среднее время ожидания: {mean_wait:.1f} мин.')

# Сохранение результатов
df.to_csv('doctor_load_optimization_results.csv', index=False)
print('Результаты сохранены в doctor_load_optimization_results.csv')